## About

In this notebook we prepare a simple solution.

Note: There is dependence on `rep` library. For its installation use `pip install rep --no-dependencies`.

In [1]:
import pandas
import numpy as np
from rep.utils import train_test_split_group
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

### Download data

* Create the folder `datasets`
* Download files from kaggle to this folder

### Read training and test files

In [2]:
data = pandas.read_csv('datasets/training.csv')
test = pandas.read_csv('datasets/test.csv')

In [3]:
data.head()

,EventID,Label,Mass,Corrected_mass,Pt,Pt_sum,Pt_min,IP_chi2,IP_chi2_sum,Flight_distance,Pseudorapidity,Track_number_PV,Tracks_number,Tracks_number_passed,Vertex_chi2,Weight
0,0,1,1525.020022,4209.730022,2354.220022,2670.780022,242.804022,9.304122,37.116622,32.554622,4.205552,1,2,1,5.833562,2.849395
1,0,1,5111.200004,5936.160004,3599.690004,5411.700004,420.789004,7.434904,1805.900004,1736.040004,3.195634,0,4,2,1.750624,2.849395
2,0,1,4799.079977,5751.109977,3929.529977,5233.069977,246.445977,7.428827,1300.089977,1301.229977,3.234167,1,4,2,6.719367,2.849395
3,0,1,4080.210007,5156.490007,4013.150007,4972.080007,536.857007,34.544807,1290.930007,1329.710007,3.215467,0,3,2,0.902209,2.849395
4,0,1,3828.379986,5094.629986,3989.839986,4860.719986,422.611986,57.246786,1512.289986,1497.489986,3.196906,0,3,2,2.682696,2.849395


### Define training features

Exclude `EventID`, `Label` and `Weight` from the features set

In [4]:
features = list(set(data.columns) - {'EventID', 'Label', 'Weight'})
features

['IP_chi2_sum',
 'Flight_distance',
 'Pt',
 'Tracks_number',
 'Pt_sum',
 'Corrected_mass',
 'Track_number_PV',
 'Pseudorapidity',
 'Tracks_number_passed',
 'Mass',
 'IP_chi2',
 'Pt_min',
 'Vertex_chi2']

### Divide training data into 2 parts
Here `train_test_split_group` function is used to divide into 2 parts to preserve secondary vertices from the same events in the same part of data (training or test). First argument should be events ids.

In [5]:
training_data, validation_data = train_test_split_group(data.EventID, data, random_state=11, train_size=0.66)

In [6]:
def prepare_data(data):
    result = data.copy()
    result["log_Pt_sum"] = np.log(result["Pt_sum"])
    result["log_Flight_distance"] = np.log(result["Flight_distance"])
    result["log_Pt"] = np.log(result["Pt"])
    result["log_Pt_min"] = np.log(result["Pt_min"])
    result["log_IP_chi2_sum"] = np.log(result["IP_chi2_sum"])
    result["log_IP_chi2"] = np.log(result["IP_chi2"])
    result["log_Corrected_mass"] = np.log(result["Corrected_mass"])
    result["log_Mass"] = np.log(result["Mass"])
    return result
    
features = features + ["log_Pt_sum", "log_Flight_distance", "log_Pt", "log_Pt_min", 
                       "log_IP_chi2_sum", "log_IP_chi2", "log_Corrected_mass", "log_Mass"] 


In [7]:
training_data = prepare_data(training_data)
validation_data = prepare_data(validation_data)

### Simple gradient boosting from `sklearn` training

We take all secondary vertices (SVs) for all events and train on them.

In [8]:
def do_fit(training_data):
    gb = GradientBoostingClassifier(learning_rate=0.1, n_estimators=200, subsample=0.8, random_state=13,
                                min_samples_leaf=200, max_depth=8, max_features=10)
    gb.fit(training_data[features], training_data.Label)

    proba = gb.predict_proba(training_data[features])

    top = pandas.DataFrame({"EventID": training_data.EventID.values,
                  "index": training_data.EventID.index,
                  "proba": proba[:,1]}).groupby("EventID").apply(lambda x: x.nlargest(3, "proba"))

    top_values = top["index"].values

    top_data = training_data.loc[top_values]

    gb2 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=200, subsample=0.8, random_state=13,
                                min_samples_leaf=50, max_depth=8, max_features=10)
    gb2.fit(top_data[features], top_data.Label)
    
    proba = gb2.predict_proba(training_data[features])

    top = pandas.DataFrame({"EventID": training_data.EventID.values,
                  "index": training_data.EventID.index,
                  "proba": proba[:,1]}).groupby("EventID").apply(lambda x: x.nlargest(3, "proba"))

    top_values = top["index"].values

    top_data = training_data.loc[top_values]

    gb3 = GradientBoostingClassifier(learning_rate=0.1, n_estimators=200, subsample=0.8, random_state=13,
                                min_samples_leaf=50, max_depth=8, max_features=10)
    gb3.fit(top_data[features], top_data.Label)
    
    return gb2, gb3

In [9]:
gb2, gb3 = do_fit(training_data)

### Prepare predictions, labels and weights for events (not for SVs!) on the cross-validation sample

In [10]:
def compute_mean(event_ids, values):
    """ fore each event computes average of given values """
    number_of_sv_in_event = np.bincount(event_ids)
    return np.bincount(event_ids, weights=values) / number_of_sv_in_event

In [11]:
def compute_max(values_event_ids, values, event_ids):
    """ fore each event computes average of given values """
    result = np.zeros(len(event_ids))
    for i, event in enumerate(event_ids):
        result[i] = np.max(values[values_event_ids==event])
    return result 

In [12]:
# predict each SV
events_ids = np.unique(validation_data.EventID)

# compute number of SVs in each event
number_of_sv_in_event = np.bincount(validation_data.EventID)

# compute predictions for events (take the mean value of predictions for SVs forming an event)
proba1 = gb2.predict_proba(validation_data[features])
proba2 = gb3.predict_proba(validation_data[features])

proba = (proba1 + proba2) / 2

events_proba = compute_max(validation_data.EventID.values, proba[:,1], events_ids)

# compute weights for events 
events_weights = compute_mean(validation_data.EventID, validation_data.Weight)[events_ids]

# compute labels for events 
events_labels = compute_mean(validation_data.EventID, validation_data.Label)[events_ids]

### ROC AUC for events (with weights) on the cross validation sample

In [13]:
roc_auc_score(events_labels, events_proba, sample_weight=events_weights)

0.96629050509855519

## Prepare submission to kaggle

In [14]:
all_data = prepare_data(data)

gb2, gb3 = do_fit(all_data)

In [15]:
test = prepare_data(test)
# predict each SV in test sample
kaggle_proba1 = gb2.predict_proba(test[features])
kaggle_proba2 = gb3.predict_proba(test[features])

kaggle_proba = (kaggle_proba1 + kaggle_proba2) / 2

kaggle_ids = np.unique(test.EventID)
# compute predictions for events (take the mean value of predictions for SVs forming an event)
kaggle_events_proba = compute_max(test.EventID.values, kaggle_proba[:, 1], kaggle_ids)

In [16]:
from IPython.display import FileLink
def create_solution(ids, proba, filename='baseline.csv'):
    """saves predictions to file and provides a link for downloading """
    pandas.DataFrame({'EventID': ids, 'Label': proba}).to_csv('datasets/{}'.format(filename), index=False)
    return FileLink('datasets/{}'.format(filename))
    
create_solution(kaggle_ids, kaggle_events_proba)

/Users/atsky/work/LHC/datasets/baseline.csv